# WAF Model Training - DistilBERT Fine-tuning

This notebook fine-tunes DistilBERT for Web Application Firewall (WAF) attack detection.

**Dataset:** notesbymuneeb/ai-waf-dataset from HuggingFace

**Model:** distilbert-base-uncased → binary classifier (benign/malicious)

---

## Setup

First, let's enable GPU and install dependencies.

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
!pip install -q transformers datasets accelerate scikit-learn

In [ ]:
# Imports
import torch
import numpy as np
from datasets import load_dataset, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Configuration

Adjust these parameters as needed:

In [ ]:
# Training Configuration
MODEL_NAME = "distilbert-base-uncased"
EPOCHS = 3
BATCH_SIZE = 16  # Increase to 32 if GPU has enough memory
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
OUTPUT_DIR = "./waf-distilbert"

## Load and Prepare Dataset

In [ ]:
# Load the WAF dataset from HuggingFace
print("Loading dataset: notesbymuneeb/ai-waf-dataset")
dataset = load_dataset("notesbymuneeb/ai-waf-dataset")

print(f"\nDataset structure:")
print(dataset)

In [ ]:
# Explore the dataset
print("Sample entries:")
for i in range(3):
    print(f"\n--- Sample {i+1} ---")
    print(f"Text: {dataset['train'][i]['text'][:200]}...")
    print(f"Label: {dataset['train'][i]['label']}")

In [ ]:
# Map labels to integers
label_map = {"benign": 0, "malicious": 1}

def map_labels(example):
    example["label"] = label_map[example["label"]]
    return example

dataset = dataset.map(map_labels)

# Cast label to ClassLabel for stratification
dataset = dataset.cast_column("label", ClassLabel(names=["benign", "malicious"]))

# Split into train/val (90/10)
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42, stratify_by_column="label")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(f"Train size: {len(train_dataset)}")
print(f"Eval size: {len(eval_dataset)}")

## Load Model and Tokenizer

In [ ]:
# Load tokenizer
print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model
print(f"Loading model: {MODEL_NAME}")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    id2label={0: "benign", 1: "malicious"},
    label2id={"benign": 0, "malicious": 1},
)

# Move model to GPU
model = model.to(device)
print(f"Model loaded on {device}")

In [ ]:
# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,  # Dynamic padding via DataCollator
    )

print("Tokenizing datasets...")
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

print(f"Tokenization complete!")
print(f"Train features: {train_dataset.column_names}")

## Training Setup

In [ ]:
# Metrics computation
def compute_metrics(eval_pred):
    """Compute accuracy, precision, recall, F1."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=50,
    warmup_ratio=0.1,
    fp16=torch.cuda.is_available(),  # Mixed precision for faster training
    report_to="none",
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Trainer ready!")

## Train the Model

This will take some time depending on dataset size and GPU.

In [ ]:
# Start training
print("="*50)
print("Starting training...")
print("="*50)

trainer.train()

## Evaluate the Model

In [ ]:
# Final evaluation
print("\n" + "="*50)
print("Final Evaluation Results")
print("="*50)

metrics = trainer.evaluate()

for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

## Save the Model

In [ ]:
# Save model and tokenizer
print(f"Saving model to {OUTPUT_DIR}")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("\nModel saved successfully!")
!ls -la {OUTPUT_DIR}

## Download the Model

Zip and download the trained model to use locally.

In [ ]:
# Zip the model directory
!zip -r waf-distilbert-model.zip {OUTPUT_DIR}

# Download (in Colab)
try:
    from google.colab import files
    files.download('waf-distilbert-model.zip')
    print("\nDownload started! Check your browser downloads.")
except ImportError:
    print("\nNot running in Colab. Model saved to: waf-distilbert-model.zip")

## Test the Model (Optional)

Quick test to verify the model works.

In [ ]:
# Test inference
from transformers import pipeline

classifier = pipeline("text-classification", model=OUTPUT_DIR, device=0 if torch.cuda.is_available() else -1)

# Test samples
test_samples = [
    "GET /api/users HTTP/1.1",
    "GET /search?q=<script>alert('xss')</script> HTTP/1.1",
    "POST /login username=admin&password=' OR '1'='1 HTTP/1.1",
    "GET /products/123 HTTP/1.1",
    "GET /admin/../../../etc/passwd HTTP/1.1",
]

print("\nTest Predictions:")
print("="*60)
for sample in test_samples:
    result = classifier(sample)[0]
    print(f"\nInput: {sample[:50]}..." if len(sample) > 50 else f"\nInput: {sample}")
    print(f"Prediction: {result['label']} (confidence: {result['score']:.4f})")

## (Optional) Push to HuggingFace Hub

If you want to share your model on HuggingFace.

In [ ]:
# Uncomment and run if you want to push to HuggingFace Hub
# from huggingface_hub import login
# login()  # Enter your HuggingFace token

# trainer.push_to_hub("your-username/waf-distilbert")
# tokenizer.push_to_hub("your-username/waf-distilbert")

---

## Usage Instructions

After downloading the model:

1. Extract `waf-distilbert-model.zip` to your project's `models/waf-distilbert` directory

2. Load the model in your application:

```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model_path = "models/waf-distilbert"
classifier = pipeline("text-classification", model=model_path)

result = classifier("GET /api/users HTTP/1.1")
print(result)  # [{'label': 'benign', 'score': 0.99}]
```

---

**Training complete!**